# Collect articles about hurricane Helene coverage

1. Use wayback Machine to scrape everything from a news website

2. Use UNC access's News Bank

## 2. News Bank pdf about "Helene" coverage

Use Selenium to download news article pdfs from news bank

keyword: helene

date: Sep 2024 (when the first 3 articles about Helene were published) - Oct 2024?

Location: USA - North Carolina

In [ ]:
# Import Module
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Scrape multiple pages

In [ ]:
# Open Chrome
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

# Open URL
url = "https://infoweb.newsbank.com/apps/news/results?hide_duplicates=2&offset=0&maxresults=60&sort=YMD_date%3AD&p=AMNEWS&t=country%3AUSA%21USA/state%3ANC%21USA%2B-%2BNorth%2BCarolina&f=advanced&val-base-0=helene&fld-base-0=alltext&bln-base-1=and&val-base-1=Sep%202024%20-%20Oct%202024&fld-base-1=YMD_date"
driver.get(url)
time.sleep(5)
# Download pdf
try:
    # For I pages
    for i in range(2):
        # Click "select all" box
        # await driver.find_element(By.CLASS_NAME, 'search-hits__select-all form-checkbox').click()
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "search-hits__select-all.form-checkbox"))).click()
        # Click next
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "pager__item.pager__item--next"))).click()
        time.sleep(1)
except Exception as e:
    print(f"Cannot select page {i}")
    print(e)
else:
    # Click download button
    driver.find_element(By.CLASS_NAME, 'actions-bar__button.actions-bar__button--download').click()
    time.sleep(1)
    # While there is "Next" button
    while True:
        # Click "Download" button in pop-up
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "u-button multidoc-download__button"))).click()
        # wait loader to appear and then disappear
        WebDriverWait(driver, 10).until(EC.element_to_be_selected((By.CLASS_NAME, 'multidoc__button-spinner')))
        WebDriverWait(driver, 10).until(EC.invisibility_of_element((By.CLASS_NAME, 'multidoc__button-spinner')))
        # Click "Next"
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "multidocs_pager_nav next"))).click()
        except:
            print("No Next found")
            break

driver.quit()





There is security check of robot. Can't pass ?

Ended up with manually downloading 5000+ articles by groups

In [1]:
import pdfplumber
import os
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re


extract text from pdf with <a href = 'https://github.com/jsvine/pdfplumber'>pdfplumber</a>

In [3]:
#   FUNC extract_text_from_pdf()
#   Input: directory (str) of pdf
#   Output: header (list) - article header information
#         article (list) - article text
def extract_text_from_pdf(dir, headers, articles):
    # header, article = [],[]
    with pdfplumber.open(dir) as pdf:
        for page in pdf.pages:
            # extract texts on each page and split texts by "OpenURL Link"
            texts = page.extract_text().split("\nOpenURL Link\n")
            # there is no "OpenURL Link" on that page
            if len(texts) == 1:
                articles[-1] += texts[0]
            else:
                headers.append(texts[0])
                articles.append(texts[1])
    return 

In [14]:
#   FUNC: decompose_header
#   Input: header (str)
#   Output: append titles, dates, newspapers, authors, and word_counts
def decompose_header(header):


    date_match = re.search(r"\b(September|October) \d{1,2}, \d{4}\b", header)
    
    date = date_match.group(0) if date_match else ""
    dates.append(date)
    loc = header.find(date)
    titles.append(header[:loc].replace("\n", ""))

    texts = header[loc:].split('\n')
    newspapers.append(texts[0][len(date)+1:].replace('| ', ""))

    # Line three
    match = re.search(r"Author: (.*?)Section: ", texts[1])
    author = match.group(1) if match else ""
    authors.append(author)
    match_word = re.search(r"(\d+)\s*Words", texts[1])
    word_count = match_word.group(1) if match_word else ""
    word_counts.append(word_count)

    return

In [18]:
headers1, articles1 = [],[]

path = 'News_Article__Fayetteville_Observer_The_NC___October_8_2024__p1 1.pdf'
with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            # extract texts on each page and split texts by "OpenURL Link"
            texts = page.extract_text()
            print(texts)

Fayetteville Observer, The (NC) - October 8, 2024 - page 1
October 8, 2024 | Fayetteville Observer, The (NC) | Fayetteville, North Carolina | Page 1
TRIPLEP ANEW INDOWS!E! NERGYE FFICIEN&T TAXF RIENDLY!
THE TRIPLE-PANE
MORE Upgrade lo our Energy Star MOST elfic1e11\w indow to earn up to $600 dollar t.:ix c;red1t.
DIFFERENCE
THAN JUST Our high-performance
WINDOWS
windows are made with - MONTHS-
NO INTEREST
RAI.El~H'SS E~'Tv AlUE three panes of glass
NO PAYMENTS
TRIP-' E-PIWE WJ,JDC,,.'9
ENfHY CO)R$ instead of just two, for maximum
$200 GIFT CARD WITH PURCHASE
benefit - for the same cost as
most leading window companies'
SGHEDULEAfREE
NO-PRESSURE QUOTE! double pane windows!
Treat your home to new
•
windows today!
910-500-6541
I!]
___ Scan~e htlp:llwindowdepotraleighJocalconsumer2.info!
NORTH CAROLINA'S OLDEST NEWSPAPER I ESTABLISHED IN 1816
•
The
TUESDAY, OCTOBER 8, 2024 I FAYOBSERVER.COM PART OF THE USA TODAY NETWORK
Hurricane
strengthens
to Category 4
John Bacon
USA TODAY
Hurricane Mil

In [ ]:
# Get all files under news_bank_pdf
directory = 'news_bank_pdf'
pdf_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".pdf")]

headers, articles = [],[]
# Extract articles and headers from pdfs
for path in pdf_paths:
    extract_text_from_pdf(path, headers, articles)

# Save articles and headers to dataframe
temp = pd.DataFrame({"header":headers,
                    "article": articles})

# temp.to_csv('articles_20241112.csv', index=False)

In [ ]:
# Decompose headers
titles, dates, newspapers, authors, word_counts = [],[],[],[],[]
temp['header'].apply(lambda x: decompose_header(x))

temp['title'] = titles
temp['date'] = dates
temp['newspaper'] = newspapers
temp['author'] = authors
temp['word_count'] = word_counts
temp.head()

# temp.to_csv("articles_20241112_cleaned.csv", index=False)

In [ ]:
def extract_text_tru_link(url):
    # Open Chrome
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)

    # Open URL
    # url = "https://infoweb.newsbank.com/apps/news/results?hide_duplicates=2&offset=0&maxresults=60&sort=YMD_date%3AD&p=AMNEWS&t=country%3AUSA%21USA/state%3ANC%21USA%2B-%2BNorth%2BCarolina&f=advanced&val-base-0=helene&fld-base-0=alltext&bln-base-1=and&val-base-1=Sep%202024%20-%20Oct%202024&fld-base-1=YMD_date"
    driver.get(url)
    time.sleep(5)

    # WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "search-hits__select-all.form-checkbox"))).click()
    driver.quit()
    return

extract_text_tru_link('https://infoweb.newsbank.com/apps/news/user/login?destination=results%3Fhide_duplicates%3D2%26offset%3D0%26maxresults%3D60%26sort%3DYMD_date%253AD%26p%3DAMNEWS%26t%3Dcountry%253AUSA%2521USA/state%253ANC%2521USA%252B-%252BNorth%252BCarolina%26f%3Dadvanced%26val-base-0%3Dhelene%26fld-base-0%3Dalltext%26bln-base-1%3Dand%26val-base-1%3DSep%25202024%2520-%2520Oct%25202024%26fld-base-1%3DYMD_date')


    

### Extract text from newspaper layout with pdfplumber

In [ ]:
import pdfplumber

with pdfplumber.open("News_Article__Fayetteville_Observer_The_NC___October_8_2024__p1.pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_text(x_tolerance=2, y_tolerance=2)  # Adjust for layout
        print(text)


### Extract text from newspaper layout with PyMuPDF

In [ ]:
import fitz  # PyMuPDF

# Open the PDF file
pdf = fitz.open("News_Article__Fayetteville_Observer_The_NC___October_8_2024__p1.pdf")

for page_num in range(pdf.page_count):
    page = pdf[page_num]
    blocks = page.get_text("blocks")  # Extract text blocks with coordinates

    # Sort blocks by y-coordinate for columnar layout
    blocks.sort(key=lambda block: (block[1], block[0]))  # y, x sorting
    for block in blocks:
        print(block[4])  # Text in each block

### Extract text from newspaper layout with Tesseract

Install <a href='https://github.com/tesseract-ocr/tesseract'>Tesseract</a> through homebrew

In [ ]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image

In [ ]:
# Convert PDF pages to images
pages = convert_from_path("News_Article__Fayetteville_Observer_The_NC___October_8_2024__p1.pdf")

for page in pages:
    # Crop or split image into columns if necessary
    text = pytesseract.image_to_string(page, config='--psm 6')  # PSM 6 for uniform block detection
    print(text)


### Extract text from newspaper layout with Layout parser

In [ ]:
import layoutparser as lp
from layoutparser.models.detectron2 import catalog
import copy
import os
import requests as requests
import layoutparser as lp
from pdf2image import convert_from_path

def load_model(
        config_path: str = 'lp://<dataset_name>/<model_name>/config',
):

    config_path_split = config_path.split('/')
    dataset_name = config_path_split[-3]
    model_name = config_path_split[-2]

    # get the URLs from the MODEL_CATALOG and the CONFIG_CATALOG 
    # (global variables .../layoutparser/models/detectron2/catalog.py)
    model_url = catalog.MODEL_CATALOG[dataset_name][model_name]
    config_url = catalog.CONFIG_CATALOG[dataset_name][model_name]

    # override folder destination:
    if 'model' not in os.listdir():
        os.mkdir('model')

    config_file_path, model_file_path = None, None

    for url in [model_url, config_url]:
        filename = url.split('/')[-1].split('?')[0]
        save_to_path = f"model/" + filename
        if 'config' in filename:
            config_file_path = copy.deepcopy(save_to_path)
        if 'model_final' in filename:
            model_file_path = copy.deepcopy(save_to_path)

        # skip if file exist in path
        if filename in os.listdir("model"):
            continue
        # Download file from URL
        r = requests.get(url, stream=True, headers={'user-agent': 'Wget/1.16 (linux-gnu)'})

        with open(save_to_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=4096):
                if chunk:
                    f.write(chunk)

    # load the label map
    label_map = catalog.LABEL_MAP_CATALOG[dataset_name]

    return lp.models.Detectron2LayoutModel(
        config_path=config_file_path,
        model_path=model_file_path,
        label_map=label_map
    )


In [ ]:
# Load the image of the PDF page (or convert PDF to image as above)
# image = Image.open("page_image.png")
pages = convert_from_path("News_Article__Fayetteville_Observer_The_NC___October_8_2024__p1.pdf")

# Error occurs: having trouble finding the model
# layout = lp.models.Detectron2LayoutModel("lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config")
# layout = lp.models.Detectron2LayoutModel("lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config")
# layout = lp.Detectron2LayoutModel(
# "./config.yaml",
# "./model_final.pth",
# extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
# label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"},
# )

# This works
# layout = load_model('lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config')


detected_layout = layout.detect(pages[0])

for block in detected_layout:
    segment = block.extract(pages)
    text = pytesseract.image_to_string(segment)  # OCR on each segment
    print(text)

## Filter news article data 

Save only the counties that experienced disasters

In [8]:
import pandas as pd
import re

In [5]:
county = pd.read_csv('helene_county_newsrooms/WNC Helene counties.csv')
county

# Clean the county name
county['County'] = county['County'].str.replace(' (County)', '')
county

,County
0,Alexander
1,Alleghany
2,Ashe
3,Avery
4,Buncombe
5,Burke
6,Cabarrus
7,Caldwell
8,Catawba
9,Cherokee


In [ ]:
news_census = pd.read_excel('helene_county_newsrooms/NC-News-Census-1.xlsx')

# Join two datasets
helene_news = pd.merge(county, news_census, how='left')

# Filter by 'newspaper' & 'digital' types only
helene_news.Type = helene_news.Type.str.lower()
helene_newspaper = helene_news[helene_news['Type'].isin(['newspaper','digital'])]
helene_newspaper.head()

# Save it locally
# helene_newspaper.to_csv('helene_newspaper.csv', index=False)

,County,Outlet,Access Area,URL,Leader,Leader Email,General Email,Owner/Group,Type
0,Alexander,Taylorsville Times,Alexander,https://www.taylorsvilletimes.com/,NaN,NaN,taylorsvilletimes@taylorsvilletimes.com,Sharpe Walter Lee,newspaper
2,Alleghany,Alleghany News,Alleghany,https://www.alleghanynews.com/,"Coby LaRue, Publisher",publisher@alleghanynews.com,news@alleghanynews.com,The Alleghany News Publishing Co. Inc,newspaper
3,Ashe,Ashe Post & Times,"Alleghany, Ashe, Avery, Mitchell, Watauga, Wil...",https://www.ashepostandtimes.com/,Teresa Laws ‚Äì General Manager/Advertising,teresa.laws@ashepostandtimes.com,moss.brennan@mountaintimes.com,Adams Publishing Group,newspaper
4,Avery,The Avery Journal-Times,Avery,https://www.averyjournal.com/,Jamie Shell,editor@averyjournal.com,news@averyjournal.com,Adams Publishing Group,newspaper
5,Buncombe,Mountain Xpress,Buncombe,https://mountainx.com/,Jeff Fobes,publisher@mountainx.com,news@mountainx.com,"Fobes, Jeff",newspaper


In [59]:
helene_newspaper = pd.read_csv('helene_newspaper.csv')
articles = pd.read_csv('articles_20241112_cleaned.csv')
articles.head()

,header,article,title,date,newspaper,author,word_count
0,"Samaritan’s Purse, Alliance Bible Fellowship e...",BOONE — Samaritan's Purse and Alliance Bible F...,"Samaritan’s Purse, Alliance Bible Fellowship e...","October 2, 2024","Watauga Democrat, The (Boone, NC)",Luke Barber luke.barber@wataugademocrat.com,593.0
1,Watch out for scammers as recovery efforts mov...,Attorney General Josh Stein declared the state...,Watch out for scammers as recovery efforts mov...,"October 2, 2024","Watauga Democrat, The (Boone, NC)",Ben McNeely,379.0
2,STORM RECOVERY: Carbon Monoxide Safety\nOctobe...,Widespread power outages due to damage from Hu...,STORM RECOVERY: Carbon Monoxide Safety,"October 2, 2024","Watauga Democrat, The (Boone, NC)",Staff Writer,440.0
3,Outage restoration continues with more than 40...,More than 300 line technicians consisting of B...,Outage restoration continues with more than 40...,"October 2, 2024","Watauga Democrat, The (Boone, NC)",Moss Brennan moss.brennan@wataugademocrat.com,267.0
4,Searches underway as Helene deaths pass 150 - ...,"SWANNANOA, N.C. - Cadaver dogs and search crew...",Searches underway as Helene deaths pass 150 - ...,"October 2, 2024",Winston-Salem Journal (NC),Associated Press,174.0


In [60]:
# FUNC: correct_news_name
# Input: newspaper name from NewsBank(str) e.g. Watauga Democrat, The (Boone, NC)
# Output: newspaper name corrected e.g. The Watauga Democrat
def correct_news_name(newspaper):
    if ", The" in newspaper:
        loc = newspaper.find(", The")
        newspaper = newspaper[:loc]
    elif "(" in newspaper:
        loc = newspaper.find('(')
        newspaper = newspaper[:loc]

    # Get rid of "The" at the beginning
    pattern = r"^The\s"
    newspaper = re.sub(pattern, "", newspaper)
    
    return newspaper

In [61]:
articles['newspaper'] = articles['newspaper'].apply(lambda x: correct_news_name(x))
articles.head()

,header,article,title,date,newspaper,author,word_count
0,"Samaritan’s Purse, Alliance Bible Fellowship e...",BOONE — Samaritan's Purse and Alliance Bible F...,"Samaritan’s Purse, Alliance Bible Fellowship e...","October 2, 2024",Watauga Democrat,Luke Barber luke.barber@wataugademocrat.com,593.0
1,Watch out for scammers as recovery efforts mov...,Attorney General Josh Stein declared the state...,Watch out for scammers as recovery efforts mov...,"October 2, 2024",Watauga Democrat,Ben McNeely,379.0
2,STORM RECOVERY: Carbon Monoxide Safety\nOctobe...,Widespread power outages due to damage from Hu...,STORM RECOVERY: Carbon Monoxide Safety,"October 2, 2024",Watauga Democrat,Staff Writer,440.0
3,Outage restoration continues with more than 40...,More than 300 line technicians consisting of B...,Outage restoration continues with more than 40...,"October 2, 2024",Watauga Democrat,Moss Brennan moss.brennan@wataugademocrat.com,267.0
4,Searches underway as Helene deaths pass 150 - ...,"SWANNANOA, N.C. - Cadaver dogs and search crew...",Searches underway as Helene deaths pass 150 - ...,"October 2, 2024",Winston-Salem Journal,Associated Press,174.0


In [ ]:
# Remove "The " at the start of the names
helene_newspaper['newspaper'] = helene_newspaper.Outlet.apply(lambda x: correct_news_name(x))
helene_newspaper.head()

,County,Outlet,Access Area,URL,Leader,Leader Email,General Email,Owner/Group,Type,newspaper
0,Alexander,Taylorsville Times,Alexander,https://www.taylorsvilletimes.com/,NaN,NaN,taylorsvilletimes@taylorsvilletimes.com,Sharpe Walter Lee,newspaper,Taylorsville Times
1,Alleghany,Alleghany News,Alleghany,https://www.alleghanynews.com/,"Coby LaRue, Publisher",publisher@alleghanynews.com,news@alleghanynews.com,The Alleghany News Publishing Co. Inc,newspaper,Alleghany News
2,Ashe,Ashe Post & Times,"Alleghany, Ashe, Avery, Mitchell, Watauga, Wil...",https://www.ashepostandtimes.com/,Teresa Laws ‚Äì General Manager/Advertising,teresa.laws@ashepostandtimes.com,moss.brennan@mountaintimes.com,Adams Publishing Group,newspaper,Ashe Post & Times
3,Avery,The Avery Journal-Times,Avery,https://www.averyjournal.com/,Jamie Shell,editor@averyjournal.com,news@averyjournal.com,Adams Publishing Group,newspaper,Avery Journal-Times
4,Buncombe,Mountain Xpress,Buncombe,https://mountainx.com/,Jeff Fobes,publisher@mountainx.com,news@mountainx.com,"Fobes, Jeff",newspaper,Mountain Xpress


In [ ]:
# Filter articles with helene newspapers with "newspaper"
h_newspapers = helene_newspaper['newspaper'].to_list()

helene_articles = articles[articles['newspaper'].isin(h_newspapers)]
# Save it locally
helene_articles.to_csv('helene_articles_20241113.csv', index=False)
len(helene_articles)

1911

In [66]:
helene_articles.head()

,header,article,title,date,newspaper,author,word_count
0,"Samaritan’s Purse, Alliance Bible Fellowship e...",BOONE — Samaritan's Purse and Alliance Bible F...,"Samaritan’s Purse, Alliance Bible Fellowship e...","October 2, 2024",Watauga Democrat,Luke Barber luke.barber@wataugademocrat.com,593.0
1,Watch out for scammers as recovery efforts mov...,Attorney General Josh Stein declared the state...,Watch out for scammers as recovery efforts mov...,"October 2, 2024",Watauga Democrat,Ben McNeely,379.0
2,STORM RECOVERY: Carbon Monoxide Safety\nOctobe...,Widespread power outages due to damage from Hu...,STORM RECOVERY: Carbon Monoxide Safety,"October 2, 2024",Watauga Democrat,Staff Writer,440.0
3,Outage restoration continues with more than 40...,More than 300 line technicians consisting of B...,Outage restoration continues with more than 40...,"October 2, 2024",Watauga Democrat,Moss Brennan moss.brennan@wataugademocrat.com,267.0
12,"Charlotte Observer, The (NC): Page 1\nOctober ...","Citation (aglc Style)\nCharlotte Observer, The...","Charlotte Observer, The (NC): Page 1","October 2, 2024",Charlotte Observer,NaN,NaN


There are 1911 articles on NewsBank from newsrooms in disaster areas of Helene from 9/23/2024 to 10/23/2024

In [70]:
len(helene_articles[helene_articles.word_count.isna()])

232

232 of them still need to be collected manually because of the image issue